# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [2]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import os
import csv

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = os.path.join('..', 'Vacation_Search', 'WeatherPy_vacation.csv')

# Display sample data
vacation_df = pd.read_csv(vacation_df)
vacation_df = vacation_df.drop("Unnamed: 0", axis=1)
vacation_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bethel,US,39.94,few clouds,41.3712,-73.4140,Hampton Inn Danbury
1,Bunia,CD,62.71,light rain,1.5667,30.2500,Hôtel Shalom
2,Cape Town,ZA,64.11,broken clouds,-33.9258,18.4232,Townhouse Hotel
3,Khatanga,RU,-9.58,broken clouds,71.9667,102.5000,Хатанга
4,Kolvereid,NO,38.43,overcast clouds,64.8655,11.6047,Bakkalandet hotell
5,Jand,PK,59.38,clear sky,33.4299,72.0193,Rehman Hotel Jand
6,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
7,Rawson,AR,57.49,few clouds,-43.3002,-65.1023,Hotel Deportivo
8,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,Apart Hotel Aires del Beagle
9,Oga,IT,28.94,overcast clouds,46.4631,10.3470,Residence Inn


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [4]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Description"]
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [5]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City']=="Oga"]
vacation_end = vacation_df.loc[vacation_df['City']=="Bjelovar"]
vacation_stop1 = vacation_df.loc[vacation_df['City']=="Bressanone"]
vacation_stop2 = vacation_df.loc[vacation_df['City']=="Hermagor"]
vacation_stop3 = vacation_df.loc[vacation_df['City']=="Zlobin"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [6]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
9,Oga,IT,28.94,overcast clouds,46.4631,10.3470,Residence Inn
298,Bressanone,IT,36.23,overcast clouds,46.7150,11.6560,Goldenes Kreuz
226,Hermagor,AT,30.96,overcast clouds,46.6272,13.3672,Bürgerbräu
250,Zlobin,HR,34.90,clear sky,45.2924,14.6541,Bitoraj
255,Bjelovar,HR,38.89,overcast clouds,45.8986,16.8489,Hotel Central


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [7]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City", "Lat", "Lng"]].copy()

# Display sample data
waypoints_df

,City,Lat,Lng
9,Oga,46.4631,10.3470
298,Bressanone,46.7150,11.6560
226,Hermagor,46.6272,13.3672
250,Zlobin,45.2924,14.6541
255,Bjelovar,45.8986,16.8489


## Use GeoViews to create map that shows the four cities in the itinerary

In [8]:
# Configure the map plot by using the itineraty_df
route_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    size = 500,
    hover_cols = "City"
)


In [9]:
# Display the route_map
route_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [10]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [11]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '3a28678f86c7411baa6958695fb1994e',
 'waypoints': '46.4631,10.347|46.715,11.656|46.6272,13.3672|45.2924,14.6541|45.8986,16.8489'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [12]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

#I used the built in geoapify route finder API doc https://apidocs.geoapify.com/playground/routing/ to see what the url
#format should be like. Below is the url they generated for me, then below that is my hard code of it. 
#geoapify_route_generation = "https://api.geoapify.com/v1/routing?waypoints=46.4630826,10.3470361|46.7165491,11.657851|46.6126272,13.355188956890357|45.2926096,14.6536287|45.8986869,16.8421977&mode=drive&apiKey=" + f"{geoapify_key}"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = str(base_url) + "?waypoints=" + str(params["waypoints"]) + "&mode=" + str(params["mode"]) + "&apiKey=" + str(geoapify_key)

# Convert the API response to JSON format
route_response_json = requests.get(route_response).json()
route_response_json

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [10.347, 46.4631], 'original_index': 0},
     {'location': [11.656, 46.715], 'original_index': 1},
     {'location': [13.3672, 46.6272], 'original_index': 2},
     {'location': [14.6541, 45.2924], 'original_index': 3},
     {'location': [16.8489, 45.8986], 'original_index': 4}],
    'units': 'metric',
    'distance': 826881,
    'distance_units': 'meters',
    'time': 34399.755,
    'legs': [{'distance': 171848,
      'time': 8241.784,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 18,
        'time': 1.88,
        'instruction': {'text': 'Drive southeast.'}},
       {'from_index': 1,
        'to_index': 15,
        'distance': 375,
        'time': 44.517,
        'instruction': {'text': 'Turn left onto Via Roma.'}},
       {'from_index': 15,
        'to_index': 43,
        'distance': 1008,
        'time': 90.812,
        'instruction': {'text': 'Bear right

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [13]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response_json['features'][0]['geometry']['coordinates']
legs

[[[10.347205, 46.463169],
  [10.347309, 46.463021],
  [10.347499, 46.463101],
  [10.348184, 46.463389],
  [10.348394, 46.463594],
  [10.34848, 46.46375],
  [10.348538, 46.463856],
  [10.348565, 46.463893],
  [10.348775, 46.464176],
  [10.348976, 46.464501],
  [10.349047, 46.464702],
  [10.349102, 46.464801],
  [10.349475, 46.465222],
  [10.349627, 46.465481],
  [10.349701, 46.46564],
  [10.349723, 46.465807],
  [10.350239, 46.466568],
  [10.35062, 46.467146],
  [10.351192, 46.468085],
  [10.351861, 46.469251],
  [10.351955, 46.469408],
  [10.352017, 46.469461],
  [10.352172, 46.469561],
  [10.352418, 46.469692],
  [10.352557, 46.469797],
  [10.352973, 46.470381],
  [10.353043, 46.470507],
  [10.353081, 46.470632],
  [10.353121, 46.470876],
  [10.353179, 46.471233],
  [10.353227, 46.471354],
  [10.353331, 46.471444],
  [10.353694, 46.47172],
  [10.353847, 46.471859],
  [10.353971, 46.472307],
  [10.354079, 46.472694],
  [10.354238, 46.473036],
  [10.354361, 46.473238],
  [10.354382, 46.

In [14]:
legs[0][0][0]

10.347205

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [15]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

row_number = 0
# Loop through the legs coordinates to fetch the latitude and longitude for each step
for row in legs[0]:
    longitude.append(legs[0][row_number][0])
    latitude.append(legs[0][row_number][1])
    row_number += 1

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [23]:
route_path

:Path   [Latitude,Longitude]

In [16]:
# Create an empty DataFrame to store the steps' coordinates

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df = pd.DataFrame({'Latitude':latitude, 'Longitude':longitude})

# Display sample data
route_df.head(10)

,Latitude,Longitude
0,46.463169,10.347205
1,46.463021,10.347309
2,46.463101,10.347499
3,46.463389,10.348184
4,46.463594,10.348394
5,46.463750,10.348480
6,46.463856,10.348538
7,46.463893,10.348565
8,46.464176,10.348775
9,46.464501,10.348976


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [24]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width=10,
    color="blue"
)

In [25]:
# Display a composed plot by using the route_map and route_path objects
route_path * route_map

:Overlay
   .Path.I   :Path   [Latitude,Longitude]
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)